In [ ]:
!python -m pip install scikit-learn

In [ ]:
!python -m pip install pandas

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
!python -m pip show scikit-learn

In [ ]:
# importing sklearn to load the Iris dataset
from sklearn.datasets import load_iris
import pandas as pd

# loading the dataset
iris = load_iris()

# Access the features and target variable
x = iris.data  # Features (sepal length, sepal width, petal length, petal width)
y = iris.target  # Labels (species: 0 for setosa, 1 for versicolor, 2 for virginica)

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

df['species'] = iris.target_names[iris.target]

df.head()

In [ ]:
df = pd.get_dummies(df, columns=['species'], dtype=int)

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print(df.columns.tolist())

# Define the feature and label column names (after one-hot encoding)
feature_columns = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
label_columns = [
    'species_setosa',
    'species_versicolor',
    'species_virginica'
]

# Extract feature data and one-hot encoded labels
X = df[feature_columns].values
y = df[label_columns].values

In [ ]:
# random_state=42 controls the random shuffling ensures we get the same split every time (reproducibility)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Standardize the features (mean=0, std=1) to improve model performance
# Standardization helps the model learn faster and more accurately
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# The learning rate controls the speed at which a neural network learns.
# Too high = overshoot
# Too low = slow learning

def create_model(learning_rate=0.01):
    model = keras.Sequential([
        keras.layers.Input(shape=(4,)),  
        # Input layer --> SLIDE NUM (18)

        keras.layers.Dense(16, activation='relu'),  
        # First hidden layer with 16 neurons --> SLIDE NUM (19)

        keras.layers.BatchNormalization(),
        # Batch Normalization --> SLIDE NUM (20) 
        # It adjusts and scales inputs to the next layer to make training faster and more stable.

        keras.layers.Dropout(0.2),
        # Dropout randomly turns off 20% of neurons during each training step.
        # This makes the model more robust by preventing overfitting.

        keras.layers.Dense(8, activation='relu'),  
        # Second hidden layer with 8 neurons
        # Fewer neurons here helps reduce complexity as we move deeper into the model.

        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),

        keras.layers.Dense(3, activation='softmax')  
        # Output layer with 3 neurons (one for each species)
        # SLIDE NUM (21)
    ])
    
    return model

In [ ]:
# Create the model with the previously defined function: create_model
learning_rate = 0.01
model = create_model(learning_rate)

# Compile the model: define how it learns (optimizer), what it learns (loss), and what to track (metrics)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate), # popular optimizer that adjusts itself
    loss='categorical_crossentropy', # best for multi-classification
    metrics=['accuracy']  # We want to track how often the model gets it right
)

# Early stopping = stop training when validation loss stops improving for 10 rounds
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

In [ ]:
# batch_size=8 = update model every 8 examples
# epochs=50 = limit to 50 training rounds (early stopping might stop sooner)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),  # Use test data to monitor performance during training
    epochs=50,
    batch_size=8,
    callbacks=[early_stopping],
    verbose=1  # 1 = show training progress
)

print("Training done!")

In [ ]:
# Evaluate the final model on the test data
# This gives us the final accuracy of our model on unseen data
print("\nEvaluating on test data:")
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# save the model
import pickle
filename = 'savedmodel.sav'
pickle.dump(model, open(filename, 'wb'))
print("Model saved successfully!")

In [ ]:
import pickle
load_model = pickle.load(open('savedmodel.sav','rb'))
print("Model loaded successfully!")

In [ ]:
!python -m pip install numpy

In [ ]:
import numpy as np

sample = np.array([[6.7, 3.3, 5, 2.1]])  # Virginica
pred_probs = load_model.predict(sample)
pred_class = np.argmax(pred_probs, axis=1)
print(pred_class)